In [1]:
!which python

/usr/local/bin/python


In [2]:
spark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
from IPython.display import display
from pyspark.sql.functions import broadcast
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .appName("hw3") \
    .getOrCreate()

# 1. Disable broadcast join
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

24/12/11 07:56:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
maps_df = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv")
medals_df = spark.read.option("header", "true").csv("/home/iceberg/data/medals.csv")
match_details_df = spark.read.option("header", "true").csv("/home/iceberg/data/match_details.csv")
matches_df = spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv")
medals_matches_players_df = spark.read.option("header", "true").csv("/home/iceberg/data/medals_matches_players.csv")

In [5]:
print("medals_df");medals_df.printSchema()
print("maps_df");maps_df.printSchema()
print("matches_df");matches_df.printSchema()
print("match_details_df");match_details_df.printSchema()
print("medals_matches_players_df");medals_matches_players_df.printSchema()

medals_df
root
 |-- medal_id: string (nullable = true)
 |-- sprite_uri: string (nullable = true)
 |-- sprite_left: string (nullable = true)
 |-- sprite_top: string (nullable = true)
 |-- sprite_sheet_width: string (nullable = true)
 |-- sprite_sheet_height: string (nullable = true)
 |-- sprite_width: string (nullable = true)
 |-- sprite_height: string (nullable = true)
 |-- classification: string (nullable = true)
 |-- description: string (nullable = true)
 |-- name: string (nullable = true)
 |-- difficulty: string (nullable = true)

maps_df
root
 |-- mapid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)

matches_df
root
 |-- match_id: string (nullable = true)
 |-- mapid: string (nullable = true)
 |-- is_team_game: string (nullable = true)
 |-- playlist_id: string (nullable = true)
 |-- game_variant_id: string (nullable = true)
 |-- is_match_over: string (nullable = true)
 |-- completion_date: string (nullable = true)
 |-- match_

In [6]:
maps_df.toPandas().head(20)

,mapid,name,description
0,c93d708f-f206-11e4-a815-24be05e24f7e,Urban,Andesia was the crucible for countless heroes ...
1,cb251c51-f206-11e4-8541-24be05e24f7e,Raid on Apex 7,This unbroken ring is a symbol of the discover...
2,c854e54f-f206-11e4-bddc-24be05e24f7e,March on Stormbreak,None
3,c8d69870-f206-11e4-b477-24be05e24f7e,Escape from A.R.C.,Scientists flocked to this Forerunner excavati...
4,73ed1fd0-45e5-4bb9-ab6a-d2852c04ea7d,Osiris,None
5,96c3e3dd-7703-4086-9e64-e3a23932bdc4,Blue Team,None
6,1c4f8e19-b046-4f78-9e2d-959cba84663d,Glassed,None
7,825065cf-df57-42e3-b845-830e7340ea43,Unconfirmed,None
8,9a188f67-1664-4d7b-83ca-1d74f714f764,Alliance,None
9,2702ea83-2c3e-4fd5-8370-60d9a6e0422f,Before the Storm,None


In [7]:
medals_df.toPandas().head(20)

,medal_id,sprite_uri,sprite_left,sprite_top,sprite_sheet_width,sprite_sheet_height,sprite_width,sprite_height,classification,description,name,difficulty
0,2315448068,None,None,None,None,None,None,None,None,None,None,None
1,3565441934,None,None,None,None,None,None,None,None,None,None,None
2,4162659350,https://content.halocdn.com/media/Default/game...,750,750,74,74,1125,899,Breakout,Kill the last enemy within the last 10 seconds...,Buzzer Beater,45
3,1573153198,https://content.halocdn.com/media/Default/game...,0,300,74,74,1125,899,Breakout,Survive a one-on-one encounter.,Vanquisher,30
4,298813630,https://content.halocdn.com/media/Default/game...,0,825,74,74,1125,899,Style,Kill an enemy with Spartan Charge.,Spartan Charge,135
5,3824002610,https://content.halocdn.com/media/Default/game...,750,600,74,74,1125,899,Vehicles,Assist a player in destroying an enemy Ghost.,Ghost Assist,140
6,3324603383,https://content.halocdn.com/media/Default/game...,225,750,74,74,1125,899,Warzone,Kill a Grunt.,Grunt Kill,60
7,979431049,https://content.halocdn.com/media/Default/game...,975,675,74,74,1125,899,Breakout,Survive a two-on-one encounter.,Bifecta,25
8,3098362934,https://content.halocdn.com/media/Default/game...,150,750,74,74,1125,899,WeaponProficiency,Kill a player in seven shots with the Carbine ...,Perfect Kill,25
9,2435743433,https://content.halocdn.com/media/Default/game...,750,225,74,74,1125,899,Warzone,Kill an enemy in a controlled Base.,Base Defense,30


In [8]:
match_details_df.toPandas().head()

24/12/11 07:56:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,match_id,player_gamertag,previous_spartan_rank,spartan_rank,previous_total_xp,total_xp,previous_csr_tier,previous_csr_designation,previous_csr,previous_csr_percent_to_next_tier,...,player_total_ground_pound_kills,player_total_shoulder_bash_kills,player_total_grenade_damage,player_total_power_weapon_damage,player_total_power_weapon_grabs,player_total_deaths,player_total_assists,player_total_grenade_kills,did_win,team_id
0,71d79b23-4143-4359-a62e-489a27597b23,taterbase,5,5,12537,13383,1,3,0,98,...,0,0,0,0,0,13,1,0,1,1
1,71d79b23-4143-4359-a62e-489a27597b23,SuPeRSaYaInG0D,18,18,131943,132557,2,3,0,2,...,0,0,0,0,0,18,2,0,0,0
2,71d79b23-4143-4359-a62e-489a27597b23,EcZachly,21,21,168811,169762,2,5,0,94,...,0,0,0,0,0,10,4,0,1,1
3,71d79b23-4143-4359-a62e-489a27597b23,johnsnake04,14,14,64073,64639,None,None,None,None,...,0,0,0,0,0,9,2,0,0,0
4,71d79b23-4143-4359-a62e-489a27597b23,Super Mac Bros,26,26,243425,244430,1,5,0,86,...,0,0,0,0,0,15,2,0,1,1


In [9]:
matches_df.toPandas().head()

,match_id,mapid,is_team_game,playlist_id,game_variant_id,is_match_over,completion_date,match_duration,game_mode,map_variant_id
0,11de1a94-8d07-4162-9f5f-d3cc753c811c,c7edbf0f-f206-11e4-aa52-24be05e24f7e,true,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,1e473914-46e4-408d-af26-178fb115de76,true,2016-02-22 00:00:00.000000,None,None,None
1,d3643e71-3e51-43e6-a200-f4a7f306ac12,cb914b9e-f206-11e4-b447-24be05e24f7e,false,d0766624-dbd7-4536-ba39-2d890a6143a9,257a305e-4dd3-41f1-9824-dfe7e8bd59e1,true,2016-02-14 00:00:00.000000,None,None,None
2,d78d2aae-36e4-48ac-a3b5-6d4d90f90ace,c7edbf0f-f206-11e4-aa52-24be05e24f7e,true,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,1e473914-46e4-408d-af26-178fb115de76,true,2016-03-24 00:00:00.000000,None,None,55e5ee2e-88df-4657-b9ae-b6ec7ca64614
3,b440069e-ec5f-4f51-bdd1-bc0bc7fe1195,c7edbf0f-f206-11e4-aa52-24be05e24f7e,true,f72e0ef0-7c4a-4307-af78-8e38dac3fdba,1e473914-46e4-408d-af26-178fb115de76,true,2015-12-23 00:00:00.000000,None,None,ec3eef73-13e3-4d4b-a922-cc195109a842
4,1dd475fc-ee6b-4e1d-8140-c44d03812076,c93d708f-f206-11e4-a815-24be05e24f7e,true,0e39ead4-383b-4452-bbd4-babb7becd82e,42f97cca-2cb4-497a-a0fd-ceef1ba46bcc,true,2016-04-07 00:00:00.000000,None,None,None


In [10]:
medals_matches_players_df.toPandas().head()

,match_id,player_gamertag,medal_id,count
0,009fdac5-e15c-47c6-a202-e18ff8800ce7,EcZachly,3261908037,7
1,009fdac5-e15c-47c6-a202-e18ff8800ce7,EcZachly,824733727,2
2,009fdac5-e15c-47c6-a202-e18ff8800ce7,EcZachly,2078758684,2
3,009fdac5-e15c-47c6-a202-e18ff8800ce7,EcZachly,2782465081,2
4,9169d1a3-955c-4ea9-a9a4-6d57da097660,EcZachly,3001183151,1


## Broadcast JOIN

In [11]:
# Perform broadcast join
result_df = medals_df.join(
    broadcast(maps_df),  # Broadcasting the smaller DataFrame (maps_df)
    on='name',  
    how='inner'
)

result_df.show(5)
result_df.explain()

+----+--------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----------+-----+-----------+
|name|medal_id|sprite_uri|sprite_left|sprite_top|sprite_sheet_width|sprite_sheet_height|sprite_width|sprite_height|classification|description|difficulty|mapid|description|
+----+--------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----------+-----+-----------+
+----+--------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----------+-----+-----------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [name#50, medal_id#40, sprite_uri#41, sprite_left#42, sprite_top#43, sprite_sheet_width#44, sprite_sheet_height#45, sprite_width#46, sprite_height#47, classification#48, description#49, difficulty#51, mapid#17, description#19]
   +- BroadcastHa

## Bucket JOIN

In [12]:
# 1. Repartition the DataFrames
num_buckets = 16

match_details_bucketed = match_details_df.repartition(num_buckets, "match_id")
matches_bucketed = matches_df.repartition(num_buckets, "match_id")
medal_matches_players_bucketed = medals_matches_players_df.repartition(num_buckets, "match_id")

# 2. Create temporary views
match_details_bucketed.createOrReplaceTempView("match_details_bucketed")
matches_bucketed.createOrReplaceTempView("matches_bucketed")
medal_matches_players_bucketed.createOrReplaceTempView("medal_matches_players_bucketed")

# 3. Perform the join
result_df = spark.sql("""
    SELECT *
    FROM match_details_bucketed md
    INNER JOIN matches_bucketed m ON md.match_id = m.match_id
    INNER JOIN medal_matches_players_bucketed mmp ON m.match_id = mmp.match_id
""")

# 4. Show results
result_df.limit(5).toPandas().head(5)

,match_id,player_gamertag,previous_spartan_rank,spartan_rank,previous_total_xp,total_xp,previous_csr_tier,previous_csr_designation,previous_csr,previous_csr_percent_to_next_tier,...,game_variant_id,is_match_over,completion_date,match_duration,game_mode,map_variant_id,match_id,player_gamertag,medal_id,count
0,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,carlitospatacon,49,49,799666,801349,2,4,0,4,...,1e473914-46e4-408d-af26-178fb115de76,None,2016-01-20 00:00:00.000000,None,None,9b1c13eb-5ca8-4695-b0da-3fe4859972b1,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,EcZachly,3261908037,10
1,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,carlitospatacon,49,49,799666,801349,2,4,0,4,...,1e473914-46e4-408d-af26-178fb115de76,None,2016-01-20 00:00:00.000000,None,None,9b1c13eb-5ca8-4695-b0da-3fe4859972b1,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,EcZachly,2078758684,2
2,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,carlitospatacon,49,49,799666,801349,2,4,0,4,...,1e473914-46e4-408d-af26-178fb115de76,None,2016-01-20 00:00:00.000000,None,None,9b1c13eb-5ca8-4695-b0da-3fe4859972b1,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,EcZachly,2763748638,1
3,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,carlitospatacon,49,49,799666,801349,2,4,0,4,...,1e473914-46e4-408d-af26-178fb115de76,None,2016-01-20 00:00:00.000000,None,None,9b1c13eb-5ca8-4695-b0da-3fe4859972b1,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,EcZachly,2782465081,1
4,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,carlitospatacon,49,49,799666,801349,2,4,0,4,...,1e473914-46e4-408d-af26-178fb115de76,None,2016-01-20 00:00:00.000000,None,None,9b1c13eb-5ca8-4695-b0da-3fe4859972b1,019411a2-67d4-4e0f-b6b3-554e3b3c5cd1,EcZachly,250435527,1


In [13]:
result_df.printSchema()

root
 |-- match_id: string (nullable = true)
 |-- player_gamertag: string (nullable = true)
 |-- previous_spartan_rank: string (nullable = true)
 |-- spartan_rank: string (nullable = true)
 |-- previous_total_xp: string (nullable = true)
 |-- total_xp: string (nullable = true)
 |-- previous_csr_tier: string (nullable = true)
 |-- previous_csr_designation: string (nullable = true)
 |-- previous_csr: string (nullable = true)
 |-- previous_csr_percent_to_next_tier: string (nullable = true)
 |-- previous_csr_rank: string (nullable = true)
 |-- current_csr_tier: string (nullable = true)
 |-- current_csr_designation: string (nullable = true)
 |-- current_csr: string (nullable = true)
 |-- current_csr_percent_to_next_tier: string (nullable = true)
 |-- current_csr_rank: string (nullable = true)
 |-- player_rank_on_team: string (nullable = true)
 |-- player_finished: string (nullable = true)
 |-- player_average_life: string (nullable = true)
 |-- player_total_kills: string (nullable = true)
 |

### Which player averages the most kills per game?

In [45]:
q1_df = result_df.groupBy("md.player_gamertag") \
            .agg(F.avg("player_total_kills").alias("avg_player_total_kills")) \
            .orderBy(F.desc("avg_player_total_kills"))

q1_df.show()

+---------------+----------------------+
|player_gamertag|avg_player_total_kills|
+---------------+----------------------+
|   gimpinator14|                 109.0|
|  I Johann117 I|                  96.0|
|BudgetLegendary|                  83.0|
|      GsFurreal|                  75.0|
|   TameablePoet|     74.22429906542057|
|   Sexy is Back|                  73.0|
|   killerguy789|                  68.0|
|THC GUILTYSPARK|                  67.0|
|    HisLattice1|                  66.0|
|PrimePromethean|                  66.0|
|     Profit TKO|     64.38181818181818|
|     taurenmonk|                  64.0|
|  DBossCnDTEXAS|     63.47586206896552|
|   Dinosaur B0B|                  63.0|
|WhiteMountainDC|                  63.0|
|        Darugis|                  62.0|
|       BlightNB|                  62.0|
|     MONKEYBAKE|                  62.0|
|    ManicZ0mb1e|                  61.0|
|Lord Leonidamir|                  60.0|
+---------------+----------------------+
only showing top

### Which playlist gets played the most?


### Which map gets played the most?


### Which map do players get the most Killing Spree medals on?